In [ ]:
import pandas as pd

aggregation_order = pd.read_csv(r"C:\Users\joech\OneDrive\workspace\HKTV OpenDataBank\RFM_project\official_draft\dataset\order_level_aggregation.csv", index_col = 0)

In [1]:
def data_cleansing(dataset):
    target_col = [
        "hktv_ad_id",
        "order_number",
        "additional_parent_order",
        "order_date",
        "order_time_range",
        "delivery_date",
        "delivery_timeslot",
        "app_type",
        "device_type",
        "sales_application",
        "payment_gateway",
        "area",
        "district",
        "housing_type",
        'order_value',
        'total_discounts',
        'credit_applied',
        'cash_voucher_code',
        'paid_voucher_code',
        'free_delivery_voucher',
        'order_mall_level_promotion_codes',
        'order_redemption_promotion_codes'
    ]
    dataset = dataset[target_col].drop_duplicates()
    membership_level_dataset = pd.DataFrame()

    # transfer data
    date_col = ["order_date", "delivery_date"]
    price_col = ["order_value"]
    promotion_col = ["hktv_ad_id", "cash_voucher_code", "paid_voucher_code", "free_delivery_voucher", "order_mall_level_promotion_codes", "order_redemption_promotion_codes"] # these are the voucher the customer would use in the check out stage

    import datetime
    def transfer_date(val):
        if str(val) == "nan" or str(val) == "NaT": return ""
        if type(val) == pd.Timestamp: return val
        return datetime.datetime.strptime(val, "%Y-%m-%d")

    def transfer_price(val):
        if type(val) == float: return val
        bins = val.replace(" ", "").split('-')
        return bins.sum()/2

    dataset.loc[:, price_col] = dataset[price_col].applymap(transfer_price)
    dataset.loc[:, date_col] = dataset[date_col].applymap(transfer_date)

    membership_level_dataset["lifetime_promotion_enjoyed"] = dataset[promotion_col].groupby("hktv_ad_id").nunique().sum(axis=1)

    def get_week_day(val):
        # convert the date to weekdays
        if str(val) == "NaT": return ""
        return val.strftime("%a")

    date_dummies = pd.get_dummies(dataset[date_col].applymap(get_week_day).add_suffix("_weekdays"))
    date_dummies['hktv_ad_id'] = dataset.hktv_ad_id
    date_dummies = date_dummies.groupby("hktv_ad_id").sum()
    membership_level_dataset = membership_level_dataset.merge(date_dummies, left_index=True, right_index=True)
    lead_time = dataset[date_col].diff(axis=1).iloc[:, 1].rename("lead_time_average")
    lead_time_mean = pd.concat([dataset.hktv_ad_id, lead_time], axis=1).groupby("hktv_ad_id").sum().applymap(lambda x: x.days)
    membership_level_dataset = membership_level_dataset.merge(lead_time_mean, left_index=True, right_index=True)
    price_col = ["hktv_ad_id", "order_value", "total_discounts", "credit_applied"]
    price_group = dataset[price_col].groupby("hktv_ad_id")
    membership_level_dataset = membership_level_dataset.merge(price_group.sum().add_prefix("total_"), left_index=True, right_index=True)
    order_col = ["hktv_ad_id", "order_number", "additional_parent_order"]
    membership_level_dataset = membership_level_dataset.merge(dataset[order_col].groupby("hktv_ad_id").nunique().add_suffix("_count"), left_index=True, right_index=True)

    dummies_col = [
        "order_time_range",
        "delivery_timeslot",
        "app_type",
        "device_type",
        "sales_application",
        "payment_gateway",
        "area",
        "district",
        "housing_type"
    ]
    dummies = pd.get_dummies(dataset[dummies_col])
    dummies['hktv_ad_id'] = dataset.hktv_ad_id
    membership_level_dataset = membership_level_dataset.merge(dummies.groupby("hktv_ad_id").sum(), left_index=True, right_index=True)

    aggre = dataset[['hktv_ad_id', 'order_number']].merge(aggregation_order, on='order_number', how='left')

    group_col = aggre.groupby("hktv_ad_id")
    membership_level_dataset = membership_level_dataset.merge(group_col.sum(), left_index=True, right_index=True)
    return membership_level_dataset

In [2]:
raw_data = r"C:\Users\joech\OneDrive\workspace\HKTV OpenDataBank\RFM_project\official_draft\dataset\Q1_raw.csv"

import tqdm
lots_data = pd.read_csv(raw_data, low_memory = False, chunksize = 100000)
data = pd.concat([data_cleansing(df) for df in tqdm.tqdm(lots_data)])

0it [00:00, ?it/s]C:\Users\joech\AppData\Local\Temp\ipykernel_16316\2697683243.py:46: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  dataset.loc[:, date_col] = dataset[date_col].applymap(transfer_date)
1it [00:09,  9.89s/it]C:\Users\joech\AppData\Local\Temp\ipykernel_16316\2697683243.py:46: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  dataset.loc[:, date_col] = dataset[date_col].applymap(transfer_date)
2it [00:18,  9.25s/it]C:\Users\joech\AppData\Local\Temp\ipykernel_16316\2697683243.py:46: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=da

In [5]:
data.groupby("hktv_ad_id").sum().to_csv(r"C:\Users\joech\OneDrive\workspace\HKTV OpenDataBank\RFM_project\official_draft\dataset\membership_level_aggregation.csv")

KeyboardInterrupt: 

In [6]:
data.groupby("hktv_ad_id").sum()

,lifetime_promotion_enjoyed,order_date_weekdays_Sat,order_date_weekdays_Sun,delivery_date_weekdays_,delivery_date_weekdays_Fri,delivery_date_weekdays_Mon,delivery_date_weekdays_Sat,delivery_date_weekdays_Sun,delivery_date_weekdays_Thu,delivery_date_weekdays_Tue,...,sub_cat_1_name_en_Home Appliances,packing_box_type_T,storage_type_T,order_date_weekdays_Mon,district_LAU FAU SHAN NT,order_date_weekdays_Tue,order_date_weekdays_Wed,order_date_weekdays_Thu,order_date_weekdays_Fri,district_SHEK KONG NT
hktv_ad_id,,,,,,,,,,,,,,,,,,,,,
+++ytBmr09h9DnF+UqtW7XmrBZooefAaYkbr7quwvL4=,0,0.0,2.0,0.0,3,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
++/XcJ8TwlaWjsE5Nu79DBhPuwXcAPNE/G4vqYXy4gY=,0,0.0,0.0,0.0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
++/o4fZf6mFzFvag92lTvQzVJNrPEgkDr2bmSOUZOAk=,0,0.0,0.0,0.0,0,0,0,0,0,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
++/zEZLENTmST6XwedyiTCkap1KRrXKHMsMH6C0kJQI=,0,0.0,0.0,0.0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
++0Fkh5hM0ez6LoMx9Bg2zKV72wB7AiFs1UVXKz/lUc=,0,0.0,0.0,0.0,0,4,0,6,0,0,...,0.0,0.0,0.0,0.0,0.0,6.0,0.0,4.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zzykG/m/rK5JpJSzZWxm3aHv6No4FIKoD4B2nDQveYI=,2,0.0,0.0,0.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
zzzVT+cTyo3QPIVViIOkJiLUOVtR56UIq9y7B/vAujw=,0,0.0,0.0,0.0,0,0,0,2,6,0,...,0.0,0.0,0.0,1.0,0.0,6.0,2.0,0.0,2.0,0.0
zzzWrZqnMZwlpQ7kgFZhaiqketpwjiZaWBn/p131A9o=,0,0.0,0.0,0.0,0,0,0,2,0,3,...,0.0,0.0,0.0,0.0,0.0,3.0,2.0,0.0,0.0,0.0
